In [ ]:
import pandas as pd
import numpy as np
import os
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import json
import requests

In [ ]:
#Read in data
myListeningDf = pd.read_csv('myListeningData.csv')

#Make some small changes for accuracy
myListeningDf["genre1"] = myListeningDf["genre1"].apply(lambda col: str(col).replace("-"," ").lower())
myListeningDf["genre2"] = myListeningDf["genre2"].apply(lambda col: str(col).replace("-"," ").lower())
myListeningDf["genre3"] = myListeningDf["genre3"].apply(lambda col: str(col).replace("-"," ").lower())
myListeningDf["genre1"] = myListeningDf["genre1"].apply(lambda col: "r n b" if (col.startswith("r") and col.endswith("b")) else col)
myListeningDf["genre2"] = myListeningDf["genre2"].apply(lambda col: "r n b" if (col.startswith("r") and col.endswith("b")) else col)
myListeningDf["genre3"] = myListeningDf["genre3"].apply(lambda col: "r n b" if (col.startswith("r") and col.endswith("b")) else col)
myListeningDf["genre1"] = myListeningDf["genre1"].apply(lambda col: col.replace("alternative rock", "alt rock"))
myListeningDf["genre2"] = myListeningDf["genre2"].apply(lambda col: col.replace("alternative rock", "alt rock"))
myListeningDf["genre3"] = myListeningDf["genre3"].apply(lambda col: col.replace("alternative rock", "alt rock"))
myListeningDf.head()

In [ ]:
#link to spotify API and get Oauth Access
clientId = os.environ.get('spotifyClientId')
clientSecret = os.environ.get('spotifyClientSecret')
clientCredentialsManager = SpotifyClientCredentials(client_id=clientId, client_secret=clientSecret)
sp = spotipy.Spotify(client_credentials_manager=clientCredentialsManager) #spotify object to access API

spotifyUsername =os.environ.get("spotifyUsername")
scope = 'playlist-modify-private playlist-modify-public'
SPOTIPY_REDIRECT_URI=os.environ.get('redirect_uri')
#Note: The order of parameters matters, and the docs were 
token = util.prompt_for_user_token(spotifyUsername,scope, clientId, clientSecret, redirect_uri=SPOTIPY_REDIRECT_URI)
if token:
    sp = spotipy.Spotify(auth=token)
    print("Success")
else:
    print("Can't get token for", username)
    
user_id = sp.current_user()["id"]

## Get All Genre Seeds

In [ ]:
#Get a list of all the possible genre seeds from which to make 'discover weekly' playlists off of
requestUrl = 'https://api.spotify.com/v1/recommendations/available-genre-seeds'
response = requests.get(requestUrl, headers={"Content-Type":"application/json", 
                        "Authorization":"Bearer "+ token}).json()['genres']
genreSeeds = [x.replace("-", " ") for x in response]
print(genreSeeds)

## Get Most Listened Genres From LastFM

In [ ]:
def mostListenedRecentGenre(df, genreSeeds):
    genreCounter = {}
    for index, row in df.iterrows():
        if row["genre1"] not in genreCounter:
            genreCounter[row["genre1"]] = 3
        else:
            genreCounter[row["genre1"]] += 3
        if row["genre2"] not in genreCounter:
            genreCounter[row["genre2"]] = 2
        else:
            genreCounter[row["genre2"]] += 2
        if row["genre3"] not in genreCounter:
            genreCounter[row["genre3"]] = 1
        else:
            genreCounter[row["genre3"]] += 1
    return sorted(genreCounter, key=genreCounter.get, reverse=True)
    

In [ ]:
#Get your 3 most listened to genres based on the data from "myListingData.csv"
genres = mostListenedRecentGenre(myListeningDf, genreSeeds)
genres.remove("nan")
genrePlaylists = 3
recommendedGenres = []
for g in genres:
    if g in genreSeeds:
        recommendedGenres.append(g)
    if len(recommendedGenres) == genrePlaylists:
        break
recommendedGenres

## Get Seed Metrics

In [ ]:
#Prepare the format of your nested dictionary for seed metrics
myListeningDf.drop(["Unnamed: 0", "id", "uri", "track_href", "analysis_url", "type"], axis = 1)
genreCols = ["genre1", "genre2", "genre3"]
seedMetrics = {}
for g in recommendedGenres: 
    seedMetrics[g] = {} #take the appropriate metrics that we think caputure the recent listening in each genre
    genreSongs = myListeningDf.loc[(myListeningDf[genreCols] == g).any(axis = 1), :]
    seedMetrics[g]["danceability"] = genreSongs["danceability"].mean()
    seedMetrics[g]["energy"] = genreSongs["energy"].mean()
    seedMetrics[g]["key"] = genreSongs["key"].mode()
    seedMetrics[g]["loudness"] = genreSongs["loudness"].mean()
    seedMetrics[g]["speechiness"] = genreSongs["speechiness"].mean()
    seedMetrics[g]["mode"] = genreSongs["mode"].mode()
    seedMetrics[g]["acousticness"] = genreSongs["acousticness"].mean()
    seedMetrics[g]["instrumentalness"] = genreSongs["instrumentalness"].mean()
    seedMetrics[g]["liveness"] = genreSongs["liveness"].mean()
    seedMetrics[g]["valence"] = genreSongs["valence"].mean()
    seedMetrics[g]["tempo"] = genreSongs["tempo"].median()
    seedMetrics[g]["duration_ms"] = genreSongs["duration_ms"].median()
    seedMetrics[g]["time_signature"] = genreSongs["time_signature"].mode()
    seedMetrics[g]["popularity"] = genreSongs["popularity"].mean()
    seedMetrics[g]["release_date"] = genreSongs["release_date"].astype('datetime64[ns]').quantile(0.5)

## Get Recommendations

In [ ]:
#Function for getting recommended tracks for each genre
def getRecommended(genre, acousticness, danceability, duration_ms, energy, instrumentalness, key, liveness, loudness, mode, popularity, speechiness, tempo, time_signature, valence, limit=50, market="US"):
    query = f'https://api.spotify.com/v1/recommendations?limit={limit}&market={market}&seed_genres={genre}&target_acousticness={acousticness}&target_danceability={danceability}&target_duration_ms={duration_ms}&target_energy={energy}&target_instrumentalness={instrumentalness}&target_key={key}&target_liveness={liveness}&target_loudness={loudness}&target_mode={mode}&target_popularity={popularity}&target_speechiness={speechiness}&target_tempo={tempo}&target_time_signature={time_signature}&target_valence={valence}'
    print(query)
    response =requests.get(query, 
                           headers={"Content-Type":"application/json", 
                                    "Authorization":"Bearer "+token}).json()
    return response
    

## Add Playlists to Spotify

In [ ]:
#For each of the genres you want a discover weekely playlist for, creates the playlist in your spotify acct
#If it doesn't work, immediately, wait for your token to expire and try again
for genre in seedMetrics:
    playlistData = getRecommended(genre, seedMetrics[genre]["acousticness"], seedMetrics[genre]["danceability"], int(seedMetrics[genre]["duration_ms"]), seedMetrics[genre]["energy"], seedMetrics[genre]["instrumentalness"], int(seedMetrics[genre]["key"]), seedMetrics[genre]["liveness"], seedMetrics[genre]["loudness"], int(seedMetrics[genre]["mode"]), int(seedMetrics[genre]["popularity"]), seedMetrics[genre]["speechiness"], seedMetrics[genre]["tempo"], int(seedMetrics[genre]["time_signature"]), seedMetrics[genre]["valence"])
    uris = []
    for x,y in enumerate(playlistData['tracks']):
            uris.append(y['uri'])
    playlist = sp.user_playlist_create(user = spotifyUsername, name = "Programmed playlist for Discover Weekly: " + genre,public = False, description="A genre specific discover weekly for " + genre)
    sp.user_playlist_add_tracks(user_id, playlist["id"], tracks = uris)
    